### To summarise tau from prediction files

In [7]:
import sys
sys.path.insert(0,
                '/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Tau_pipeline/Tau_quantification/')

from helper_functions import *

**OBEJECTIVES:**

**1) To create summary tau density:**
    output = df_subset

**2) To merge tau quantification with semi-quantitative data**

**Importing libraries**

In [8]:
import pandas as pd
import numpy as np 


### 1) To create summary tau density file

#### Import annotation area

In [9]:
c1_area = extract_area('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Annotation_measurement/cortical1_measurements.csv',
            ["Stroma"])

c2_area = extract_area('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Annotation_measurement/cortical2_measurements.csv',
            ["Stroma"])

c3_area = extract_area('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Annotation_measurement/cortical3_measurements.csv',
            ["Stroma"])
cortical_area_ = pd.concat([c1_area, c2_area, c3_area])
cortical_area = cortical_area_.copy()
cortical_area.loc[:,'Name'] = ['Cortical']*cortical_area.shape[0]
cortical_area = cortical_area.reset_index(drop=True)

In [10]:
bg_area = extract_area('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Annotation_measurement/BG_measurements.csv',
            ["STN","GP","STR"])

In [11]:
dn_area_ = extract_area('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Annotation_measurement/DN_measurements.csv',
            ["DN+WM"])
dn_area = dn_area_.copy()
dn_area.loc[:,'Name'] = ['DN']*dn_area.shape[0]

**Import prediction files**

In [12]:
cortical_files = extract_files('C:/Users/mokur/OneDrive/Desktop/Digital_path/Predictions/Cortical/',
                              'predictions.txt')
bg_files = extract_files('C:/Users/mokur/OneDrive/Desktop/Digital_path/Predictions/BG/',
                              'predictions.txt')
dn_files = extract_files('C:/Users/mokur/OneDrive/Desktop/Digital_path/Predictions/DN/',
                              'predictions.txt')


Read in:  222 files
Extracted: 222 files
Read in:  27 files
Extracted: 27 files
Read in:  32 files
Extracted: 32 files


**Calculate tau counts & total area** 

In [13]:
cortical_tau1 = calculate_tau_metrics(cortical_files,'Cortical')
bg_tau1 = calculate_tau_metrics_BG(bg_files)
dn_tau1 = calculate_tau_metrics(dn_files,'DN')

**Calculate tau density & area fraction**

In [14]:
cortical_tau2 = calculate_density_areafrac(cortical_tau1, cortical_area)
print(cortical_tau2.shape)
bg_tau2 = calculate_density_areafrac(bg_tau1, bg_area)
print(bg_tau2.shape)
dn_tau2 = calculate_density_areafrac(dn_tau1, dn_area)
print(dn_tau2.shape)

(222, 31)
(81, 31)
(32, 31)


**Combine results together**

In [15]:
tau_obj = pd.concat([
                    cortical_tau2,
                    bg_tau2,
                    dn_tau2
                    ])
print(tau_obj.shape)

(335, 31)


**Import slide metadata**

In [16]:
metadata = pd.read_csv('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Metadata/PSP_tau_classification_MAY2023.txt',
                    sep = '\t')
print(metadata.shape)
metadata.head()
print(metadata['Comments'].value_counts())

(304, 13)
Novel                   182
subcoritcal              60
No_staging               23
tau_training             10
AD_pathology             10
excluded                  6
screen_training_only      3
cortical_test             2
Excluded                  2
Cannot_locate             1
Name: Comments, dtype: int64


**Merge obj to slide metadata**

In [17]:
obj_meta = metadata.merge(tau_obj,
                          on=['Slice_ID'],
                          how='left',
                          indicator=True)
print(obj_meta.shape)
obj_meta.head()

(358, 44)


,Comments,Diagnosis,Extra_comments,PSP Stage,Patient_ID,Slice_ID,brain_hemi,brain_region,label_name,new_sample_ID,...,Others_density,Ambiguous_density,Total_tau_AF,Tau_hallmarks_AF,NFT_AF,CB_AF,TA_AF,Others_AF,Ambiguous_AF,_merge
0,Novel,PSP,Cortical,2_01,NP19-00221,S747324,L,BA46,NP19-00221_A6-2_AT8_TAU(N)_BA46,NP19-00221_A6-2_BA46_TAU,...,1.722254e-06,3.363777e-08,0.000038,0.000016,7.636190e-07,0.000006,0.000009,0.000022,0.000003,both
1,Novel,PSP,Cortical,2_01,NP19-00221,S747390,L,BA1718,NP19-00221_A7-2_AT8_TAU(N)_BA1718,NP19-00221_A7-2_BA1718_TAU,...,8.374336e-07,9.737600e-09,0.000011,0.000003,0.000000e+00,0.000003,0.000000,0.000008,0.000001,both
2,Novel,PSP,Cortical,2_01,NP19-00221,S760051,L,FCF,NP19-00221-A3-2--TAU,NP19-00221_A3-2_FCF_TAU,...,3.940675e-06,7.036920e-08,0.000075,0.000005,0.000000e+00,0.000004,0.000001,0.000070,0.000007,both
3,Novel,PSP,Cortical,2_01,NP19-00221,S760052,L,TCF,NP19-00221-A4-2--TAU,NP19-00221_A4-2_TCF_TAU,...,9.630612e-06,1.901515e-07,0.000194,0.000069,1.269558e-05,0.000033,0.000023,0.000125,0.000016,both
4,Novel,PSP,Cortical,2_01,NP19-00221,S760053,L,PCF,NP19-00221-A5-2--TAU,NP19-00221_A5-2_PCF_TAU,...,2.808938e-06,4.826353e-08,0.000066,0.000026,1.115379e-06,0.000016,0.000009,0.000040,0.000004,both


In [18]:
obj_meta['_merge'].value_counts()

both          335
left_only      23
right_only      0
Name: _merge, dtype: int64

***We are missing 23 files but after looking below, its all fine.***

In [19]:
obj_meta[obj_meta['_merge']=='left_only'][['Comments','Extra_comments','PSP Stage','Diagnosis','Slice_ID','Patient_ID','region_name']]

,Comments,Extra_comments,PSP Stage,Diagnosis,Slice_ID,Patient_ID,region_name
132,Novel,Cortical,5_05,PSP,Unknown,NP17-00245,premo motor
136,excluded,Cortical,5_07,PSP,S747384,NP18-00149,occipital
213,No_staging,Cortical,x_0,PSP,S760045,NP19-00106,temporal
228,AD_pathology,AD_pathology,3_02,PSP,S747874,NP18-00004,cerebellum
239,NaN,Cannot_locate,NaN,NaN,S771999,NP18-00321,NaN
240,NaN,Cannot_locate,NaN,NaN,S772000,NP18-00321,NaN
241,NaN,Cannot_locate,NaN,NaN,S772001,NP18-00321,NaN
261,No_staging,No_staging,x_0,PSP,S747360,NP19-00106,cerebellum
262,No_staging,No_staging,x_01,PSP,S747353,NP19-00018,cerebellum
263,NaN,No_staging,NaN,NaN,S771833,NP19-00018,NaN


**Looking at those with predictions**

In [20]:
df = obj_meta[obj_meta['_merge']=='both']

In [21]:
print(df.shape)
df['Comments'].value_counts()

(335, 44)


Novel                   181
subcoritcal              98
tau_training             18
No_staging               17
AD_pathology              8
excluded                  5
screen_training_only      3
cortical_test             2
Excluded                  2
Cannot_locate             1
Name: Comments, dtype: int64

**Exclude some predicted slides that cannot be used**

In [22]:
to_exclude =['No_staging',
            'AD_pathology',
            'excluded',
            'Excluded',
            'Cannot_locate']
df_subset_ = df[~(df['Comments'].isin(to_exclude))]
df_subset = df_subset_[~df_subset_['Extra_comments'].isin(to_exclude)] 
df_subset.shape
df_subset.head()
# print(df_subset['Comments'].value_counts())
# print(df_subset['Extra_comments'].value_counts())


,Comments,Diagnosis,Extra_comments,PSP Stage,Patient_ID,Slice_ID,brain_hemi,brain_region,label_name,new_sample_ID,...,Others_density,Ambiguous_density,Total_tau_AF,Tau_hallmarks_AF,NFT_AF,CB_AF,TA_AF,Others_AF,Ambiguous_AF,_merge
0,Novel,PSP,Cortical,2_01,NP19-00221,S747324,L,BA46,NP19-00221_A6-2_AT8_TAU(N)_BA46,NP19-00221_A6-2_BA46_TAU,...,1.722254e-06,3.363777e-08,0.000038,0.000016,7.636190e-07,0.000006,0.000009,0.000022,0.000003,both
1,Novel,PSP,Cortical,2_01,NP19-00221,S747390,L,BA1718,NP19-00221_A7-2_AT8_TAU(N)_BA1718,NP19-00221_A7-2_BA1718_TAU,...,8.374336e-07,9.737600e-09,0.000011,0.000003,0.000000e+00,0.000003,0.000000,0.000008,0.000001,both
2,Novel,PSP,Cortical,2_01,NP19-00221,S760051,L,FCF,NP19-00221-A3-2--TAU,NP19-00221_A3-2_FCF_TAU,...,3.940675e-06,7.036920e-08,0.000075,0.000005,0.000000e+00,0.000004,0.000001,0.000070,0.000007,both
3,Novel,PSP,Cortical,2_01,NP19-00221,S760052,L,TCF,NP19-00221-A4-2--TAU,NP19-00221_A4-2_TCF_TAU,...,9.630612e-06,1.901515e-07,0.000194,0.000069,1.269558e-05,0.000033,0.000023,0.000125,0.000016,both
4,Novel,PSP,Cortical,2_01,NP19-00221,S760053,L,PCF,NP19-00221-A5-2--TAU,NP19-00221_A5-2_PCF_TAU,...,2.808938e-06,4.826353e-08,0.000066,0.000026,1.115379e-06,0.000016,0.000009,0.000040,0.000004,both


**Collapsing region_name & Name columns**

In [23]:
# cortical 
cortical = df_subset[df_subset['Name']=='Cortical']
cortical = cortical.drop(columns=['Name'])

# basal ganglia 
bg_ = df_subset[df_subset['region_name']=='basal ganglia']
bg = bg_.copy()
bg.loc[:,'region_name']= bg_['Name']
bg = bg.drop(columns=['Name'])

#dentate nucelus 
dn_ = df_subset[df_subset['region_name']=='cerebellum']
dn = dn_.copy()
dn.loc[:,'region_name']= ['Dentate_nucleus']*dn_.shape[0]
dn = dn.drop(columns=['Name'])

# Combined
df_subset_formatted = pd.concat([cortical, bg, dn])
print(set(df_subset_formatted['region_name']))
print('before formatting: ', df_subset.shape)
print('formatted:', df_subset_formatted.shape)

{'frontal', 'premo motor', 'STR', 'parietal', 'GP', 'temporal', 'cingulate', 'primary somatosensory', 'STN', 'Dentate_nucleus', 'occipital', 'primary motor'}
before formatting:  (301, 44)
formatted: (301, 43)


### 2) To merge tau quantification with semi-quantitative data

In [24]:
semi_quan = pd.read_csv('/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Metadata/Pathology_assessment_and_demography.csv',sep=',')
semi_quan.head()

,REDCap ID,Patient_ID,JR Number,JR,GP_SK,STN_SK,STR_SK,FCF_SK,DE_SK,OC_SK,...,Age at Death,disease duration (years),MDS_first,MDS-PSP last visit,years PSPRS and death,years ACER and death,PSPRS Total (last assesment),ACE-R last assesment,imputed ACER at average,imputed PSPRS at death_average
0,636.0,NP19-00221,133.0,JR133,2.0,2.0,2.0,0.0,1.0,0.0,...,76.4,8.750000,prob. PSP-RS,prob. PSP-RS,0.32,0.32,63.0,94.0,86.20,62.60
1,3088.0,NP20-00012,139.0,JR139,3.0,3.0,2.0,0.0,1.0,0.0,...,75.2,4.619178,poss. PSP-PGF,poss. PSP-PGF,0.52,0.52,26.0,97.0,93.49,28.18
2,138.0,NP17-00242,94.0,JR94,2.0,2.0,2.0,1.0,1.0,0.0,...,55.0,5.500000,prob. PSP-RS,prob. PSP-RS,0.41,0.41,53.0,77.0,78.14,52.73
3,2437.0,NP18-00007,99.0,JR99,2.0,2.0,2.0,1.0,1.0,0.0,...,74.5,6.000000,prob. PSP-RS,prob. PSP-RS,0.67,0.67,45.0,79.0,75.23,55.98
4,453.0,NP18-00004,98.0,JR98,2.0,2.0,2.0,1.0,1.0,0.0,...,84.7,5.580000,prob. PSP-RS,prob. PSP-RS,1.25,1.25,59.0,66.0,71.44,58.97


In [25]:
# Check out predicted slides 
print('objective tau shape: ', df_subset_formatted.shape)
print('unique subjects from objective: ',len(list(set(df_subset_formatted['Patient_ID']))))
print('------')

# Check semi-quan data
print('semi quan shape: ', semi_quan.shape)
print('Number of unique subjects (semi): ', len(set(semi_quan['Patient_ID'])))
print('------')

# Merge obj_meta to semi-quantitative scoring 
combined = df_subset_formatted.merge(semi_quan,on=['Patient_ID'],how='left')
print('combined shape: ',combined.shape)
print('combined unique subjects: ', len(set(combined['Patient_ID'])))
print('------')

# Check if any PSP subject is missing semi-quantitative scoring 
combined_no_na = combined[combined['Stage_SK'].notna()]
print('combined after NA stage_SK removed: ',combined_no_na.shape)
print('Number of unique subjects (semi&obj): ', len(set(combined_no_na['Patient_ID'])))
print('------')

# Remove slide with duplicated regions 
output_no_dup = combined_no_na.drop_duplicates(subset=['Patient_ID','region_name'],keep='last')
output_no_dup['region_name'].value_counts()
print('No duplicated regions for each subject: ', output_no_dup.shape)

objective tau shape:  (301, 43)
unique subjects from objective:  32
------
semi quan shape:  (49, 22)
Number of unique subjects (semi):  49
------
combined shape:  (301, 64)
combined unique subjects:  32
------
combined after NA stage_SK removed:  (301, 64)
Number of unique subjects (semi&obj):  32
------
No duplicated regions for each subject:  (296, 64)


In [26]:
output_no_dup.head()

,Comments,Diagnosis,Extra_comments,PSP Stage,Patient_ID,Slice_ID,brain_hemi,brain_region,label_name,new_sample_ID,...,Age at Death,disease duration (years),MDS_first,MDS-PSP last visit,years PSPRS and death,years ACER and death,PSPRS Total (last assesment),ACE-R last assesment,imputed ACER at average,imputed PSPRS at death_average
2,Novel,PSP,Cortical,2_01,NP19-00221,S760051,L,FCF,NP19-00221-A3-2--TAU,NP19-00221_A3-2_FCF_TAU,...,76.4,8.75,prob. PSP-RS,prob. PSP-RS,0.32,0.32,63.0,94.0,86.2,62.6
3,Novel,PSP,Cortical,2_01,NP19-00221,S760052,L,TCF,NP19-00221-A4-2--TAU,NP19-00221_A4-2_TCF_TAU,...,76.4,8.75,prob. PSP-RS,prob. PSP-RS,0.32,0.32,63.0,94.0,86.2,62.6
4,Novel,PSP,Cortical,2_01,NP19-00221,S760053,L,PCF,NP19-00221-A5-2--TAU,NP19-00221_A5-2_PCF_TAU,...,76.4,8.75,prob. PSP-RS,prob. PSP-RS,0.32,0.32,63.0,94.0,86.2,62.6
5,screen_training_only,PSP,Cortical,2_01,NP19-00221,S760054,L,OCF,NP19-00221-A8-2--TAU,NP19-00221_A8-2_OCF_TAU,...,76.4,8.75,prob. PSP-RS,prob. PSP-RS,0.32,0.32,63.0,94.0,86.2,62.6
6,Novel,PSP,Cortical,2_01,NP19-00221,S760055,L,CING,NP19-00221-A9-2--TAU,NP19-00221_A9-2_CING_TAU,...,76.4,8.75,prob. PSP-RS,prob. PSP-RS,0.32,0.32,63.0,94.0,86.2,62.6


In [27]:
set(output_no_dup['region_name'])

{'Dentate_nucleus',
 'GP',
 'STN',
 'STR',
 'cingulate',
 'frontal',
 'occipital',
 'parietal',
 'premo motor',
 'primary motor',
 'primary somatosensory',
 'temporal'}

**Compare to the file this should match (created as a pilot)**

In [28]:
orig = pd.read_csv('/Users/mokur/OneDrive - University of Cambridge/Attachments/May2022/Final/Pathology/tau_classification/Dentate_nucleus/analysis/objective_semiquan_tau_PSP_cortical_bg_dn_withArea.txt', sep='\t')
orig.shape

(296, 59)

In [29]:
set(orig['region_name'])

{'Dentate_nucleus',
 'GP',
 'STN',
 'STR',
 'cingulate',
 'frontal',
 'occipital',
 'parietal',
 'premo motor',
 'primary motor',
 'primary somatosensory',
 'temporal'}

In [30]:
orig_subset = orig[['Patient_ID','Slice_ID','region_name']]
print(orig_subset.shape)
new_subset = output_no_dup[['Patient_ID','Slice_ID','region_name']]
print(new_subset.shape)

(296, 3)
(296, 3)


In [31]:
tgt = new_subset.merge(orig_subset,on=['Patient_ID','Slice_ID','region_name'], how='left', indicator=True)
tgt['_merge'].value_counts()


both          296
left_only       0
right_only      0
Name: _merge, dtype: int64

**Remove 19 slide used during model development & 2 dodgy slides**

In [32]:
# Remove dodgy slides
output_no_dup_clean = output_no_dup[(output_no_dup['Slice_ID']!='S747296')&(output_no_dup['Slice_ID']!='S747857')]

In [33]:
output_no_dup_clean['Comments'].value_counts()

Novel                   177
subcoritcal              90
tau_training             18
screen_training_only      3
cortical_test             2
Name: Comments, dtype: int64

In [34]:
# Remove those used during training
novel = output_no_dup_clean[(output_no_dup_clean['Comments']!='screen_training_only') & (output_no_dup_clean['Comments']!='tau_training')]

In [35]:
print(len(list(set(output_no_dup['Slice_ID'])))) # still contains dodgy slides
print(len(list(set(output_no_dup_clean['Slice_ID'])))) # no dodgy slide but still has training data 
print(len(list(set(novel['Slice_ID'])))) # no dodgy slides, no training slides

242
240
227


**Exporting out**

In [36]:
output_no_dup_clean.columns

Index(['Comments', 'Diagnosis', 'Extra_comments', 'PSP Stage', 'Patient_ID',
       'Slice_ID', 'brain_hemi', 'brain_region', 'label_name', 'new_sample_ID',
       'region_name', 'sample_ID', 'stained_protein', 'Non_tau', 'Others',
       'CB', 'NFT', 'Ambiguous', 'TA', 'Non_tau_area', 'Others_area',
       'CB_area', 'NFT_area', 'Ambiguous_area', 'TA_area', 'Area µm^2',
       'Total_tau', 'Total_tau_hallmarks', 'Total_tau_density',
       'Tau_hallmark_density', 'NFT_density', 'CB_density', 'TA_density',
       'Others_density', 'Ambiguous_density', 'Total_tau_AF',
       'Tau_hallmarks_AF', 'NFT_AF', 'CB_AF', 'TA_AF', 'Others_AF',
       'Ambiguous_AF', '_merge', 'REDCap ID', 'JR Number', 'JR', 'GP_SK',
       'STN_SK', 'STR_SK', 'FCF_SK', 'DE_SK', 'OC_SK', 'Stage_SK', 'Gender',
       'Age at Death', 'disease duration (years)', 'MDS_first',
       'MDS-PSP last visit', 'years PSPRS and death', 'years ACER and death',
       'PSPRS Total (last assesment)', 'ACE-R last assesment',
  

In [37]:
# For neuroimaging-path: contains training slides (but no dodgy slide)
path = '/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Tau_pipeline/Tau_quantification/Results/objective_semiquan_tau_PSP_May2023_corticalUpdated3_noArtefacts.txt'
output_no_dup_clean.to_csv(path, sep='\t',index=False)

In [38]:
# For validation: no training slides,  no dodgy slide
path = '/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Tau_pipeline/Tau_quantification/Results/objective_semiquan_tau_PSP_May2023_corticalUpdated3_noArtefacts_novel.txt'
novel.to_csv(path, sep='\t',index=False)

**Checking slide per brain**

In [39]:
patient_brain = output_no_dup_clean[['Patient_ID','Slice_ID']]
patient_brain_nodup = patient_brain.drop_duplicates(subset=['Slice_ID'])
print(patient_brain.shape)
print(patient_brain_nodup.shape)

(290, 2)
(240, 2)


In [40]:
from scipy.stats import skew, iqr
import statistics

In [41]:
print(skew(list(patient_brain_nodup['Patient_ID'].value_counts())))
print(iqr(list(patient_brain_nodup['Patient_ID'].value_counts())))

-0.687448961133831
6.0


In [42]:
print(statistics.median(list(patient_brain_nodup['Patient_ID'].value_counts())))
print(np.mean(list(patient_brain_nodup['Patient_ID'].value_counts())))
print(np.std(list(patient_brain_nodup['Patient_ID'].value_counts())))

8.5
7.5
2.704163456597992


In [43]:
patient_brain_nodup['Patient_ID'].value_counts()

NP19-00059    10
NP17-00239    10
NP21-019      10
NP18-00257    10
NP17-00082    10
NP20-00040    10
NP17-00136    10
NP17-00242    10
NP19-00055    10
NP18-00163    10
NP21-011      10
NP19-00007    10
NP17-00245     9
NP20-00012     9
NP17-00109     9
NP20-00025     9
NP18-00149     8
NP18-00007     8
NP19-00221     8
NP19-00175     8
NP18-00212     7
NP18-00291     7
NP19-00189     6
NP15-00197     4
NP18-00200     4
NP18-00321     4
NP18-00229     4
NP16-00069     4
NP16-00042     4
NP15-00272     4
NP18-00019     2
NP15-00293     2
Name: Patient_ID, dtype: int64